<a href="https://colab.research.google.com/github/davidraamirez/GradientWithoutBackpropagation/blob/main/CNN_fwd_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gradient Without Backpropagation

In [72]:
import torch
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import tqdm
import torch.distributions as distr
import numpy as np

In [2]:
%pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 4.0 MB/s eta 0:00:00


In [3]:
import torchmetrics
import torchvision
from torchvision import transforms as T
from matplotlib import pyplot as plt

Loading and preprocessing the dataset

In [4]:
#Load the dataset
train_data = torchvision.datasets.KMNIST('./data', train=True, download=True)

  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting ./data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/KMNIST/raw



In [5]:
# This loads data with both data conversion.
train_data = torchvision.datasets.KMNIST('./data', train=True, transform=T.ToTensor())

In [6]:
# Loaders are used to shuffle, batch, and possibly sample the elements of the dataset
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True)

In [24]:
Xtrain = torch.randn(500, 1, 1, 28, 28)
ytrain = torch.randn(500, 1)
for i in range(500):
  xb, yb = next(iter(train_loader))
  Xtrain[i] = xb
  ytrain[i] = yb

print(Xtrain.dtype)
print(ytrain.dtype)

torch.float32
torch.float32


In [8]:
# Loading the test data is similar, but (a) we do not apply data augmentation,
# and (b) we do not shuffle when building the mini-batches.
test_data = torchvision.datasets.KMNIST('./data', train=False, transform=T.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, shuffle=False)

In [9]:
Xtest = torch.randn(50, 1, 1, 28, 28)
ytest = torch.randn(50, 1)
for i in range(50):
  xb, yb = next(iter(test_loader))
  Xtest[i] = xb
  ytest[i] = yb

print(Xtest.shape)
print(ytest.shape)

torch.Size([50, 1, 1, 28, 28])
torch.Size([50, 1])


Define Convolutional Neural Network Class

In [10]:
from torch import nn
from torch.nn import functional as F

In [69]:
class SimpleCNN(nn.Module):
    def __init__(self, input_size, conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b,):
        super().__init__()
        self.conv1 = nn.Conv2d(input_size, 2, 3, padding=1)
        self.conv1.weight = torch.nn.Parameter(conv1w)
        self.conv1.bias = torch.nn.Parameter(conv1b)

        self.conv2 = nn.Conv2d(2, 4, 3, padding=1)
        self.conv2.weight = torch.nn.Parameter(conv2w)
        self.conv2.bias = torch.nn.Parameter(conv2b)

        self.max_pool = nn.MaxPool2d(2)

        self.w1 = torch.nn.Parameter(fc1w)
        self.b1 = torch.nn.Parameter(fc1b)

        self.w2 = torch.nn.Parameter(fc2w)
        self.b2 = torch.nn.Parameter(fc2b)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        x = x.reshape((-1, 4*14*14))
        x = F.relu(x@self.w1 + self.b1)
        x = x@self.w2 + self.b2
        return torch.softmax(x,1)

In [70]:
# We check if CUDA is available. If you do not see it,
# activate a GPU from Runtime >> Change runtime type and 
# restart the notebook.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Initialize the parameters

In [71]:
# We initialize the parameters randomly and the model with an input size
conv1w = torch.FloatTensor(2, 1, 3, 3).uniform_(-1, 1)
conv1b = torch.FloatTensor(2).uniform_(-1, 1)
conv2w = torch.FloatTensor(4, 2, 3, 3).uniform_(-1, 1)
conv2b = torch.FloatTensor(4).uniform_(-1, 1)

fc1w = torch.FloatTensor(4*14*14,8).uniform_(-1,1)
fc1b = torch.FloatTensor(8).uniform_(-1, 1)
fc2w = torch.FloatTensor(8,10).uniform_(-1, 1)
fc2b = torch.FloatTensor(10).uniform_(-1, 1)
cnn = SimpleCNN(1, conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b).to(device)

In [94]:
def conv(input, weight, bias):
  def forward_pass(self, a_prev: np.array, training: bool) -> np.array:
        """
        :param a_prev - 4D tensor with shape (n, h_in, w_in, c)
        :output 4D tensor with shape (n, h_out, w_out, n_f)
        ------------------------------------------------------------------------
        n - number of examples in batch
        w_in - width of input volume
        h_in - width of input volume
        w_out - width of input volume
        h_out - width of input volume
        c - number of channels of the input volume
        n_f - number of filters in filter volume
        """
        _a_prev = np.array(a_prev, copy=True)
        output_shape = calculate_output_dims(input_dims=a_prev.shape)
        n, h_in, w_in, _ = a_prev.shape
        _, h_out, w_out, _ = output_shape
        h_f, w_f, _, n_f =w.shape
        pad = self.calculate_pad_dims()
        a_prev_pad = self.pad(array=a_prev, pad=pad)
        output = np.zeros(output_shape)

        for i in range(h_out):
            for j in range(w_out):
                h_start = i * self._stride
                h_end = h_start + h_f
                w_start = j * self._stride
                w_end = w_start + w_f

                output[:, i, j, :] = np.sum(
                    a_prev_pad[:, h_start:h_end, w_start:w_end, :, np.newaxis] *
                    self._w[np.newaxis, :, :, :],
                    axis=(1, 2, 3)
                )

        return output + self._bç
  return F.conv2d(F.pad(input, (1, 1)),weight, bias)

In [95]:
print(conv(Xtrain[0], conv1w, conv1b))

tensor([[[[-0.9165, -0.9165, -0.9165,  ..., -0.9165, -0.9165, -0.9165],
          [-0.9165, -0.9165, -0.9165,  ..., -0.9165, -0.9165, -0.9165],
          [-0.9165, -0.9165, -0.9165,  ..., -0.9165, -0.9165, -0.9165],
          ...,
          [-0.9165, -0.9165, -0.9165,  ..., -0.9165, -0.9165, -0.9165],
          [-0.9165, -0.9165, -0.9165,  ..., -0.9165, -0.9165, -0.9165],
          [-0.9165, -0.9165, -0.9165,  ..., -0.9165, -0.9165, -0.9165]],

         [[-0.4653, -0.4653, -0.4653,  ..., -0.4653, -0.4653, -0.4653],
          [-0.4653, -0.4653, -0.4653,  ..., -0.4653, -0.4653, -0.4653],
          [-0.4653, -0.4653, -0.4653,  ..., -0.4653, -0.4653, -0.4653],
          ...,
          [-0.4653, -0.4653, -0.4653,  ..., -0.4653, -0.4653, -0.4653],
          [-0.4653, -0.4653, -0.4653,  ..., -0.4653, -0.4653, -0.4653],
          [-0.4653, -0.4653, -0.4653,  ..., -0.4653, -0.4653, -0.4653]]]])


In [84]:
print(forward_pass(Xtrain[0], conv1w, conv1b))

torch.Size([1, 1, 1, 28, 1])
torch.Size([1, 2, 1, 3, 3])


RuntimeError: ignored

In [41]:
# Note: we also need to move data when asking for a prediction
print(cnn(xb.to(device)).shape)

torch.Size([1, 10])


In [42]:
def cross_entropycw1(conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()


In [43]:
def cross_entropycb1(conv1b, conv1w, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [44]:
def cross_entropycw2(conv2w, conv1b, conv1w, conv2b, fc1w, fc1b, fc2w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [45]:
def cross_entropycb2(conv2b, conv1b, conv2w, conv1w, fc1w, fc1b, fc2w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [46]:
def cross_entropyfcw1(fc1w, conv1b, conv2w, conv2b, conv1w, fc1b, fc2w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [47]:
def cross_entropyfcb1(fc1b, conv1b, conv2w, conv2b, fc1w, conv1w, fc2w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [48]:
def cross_entropyfcw2(fc2w, conv1b, conv2w, conv2b, fc1w, fc1b, conv1w, fc2b, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [49]:
def cross_entropyfcb2(fc2b, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, conv1w, ytrue, x):
  """ Cross-entropy loss.
  Inputs:
  - ytrue (n,): vector of indices for the correct class.
  - ypred (n, 3): predictions of the model.
  Returns the average cross-entropy.
  """
  # This is called integer array indexing in NumPy:
  # https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing
  ypred=torch.randn((ytrue.shape[0],10))
  for j in range (ytrue.shape[0]):
    xj =x[j]
    conv1 = nn.Conv2d(1, 2, 3, padding=1)
    conv1.weight = torch.nn.Parameter(conv1w)
    conv1.bias = torch.nn.Parameter(conv1b)
    conv2 = nn.Conv2d(2, 4, 3, padding=1)
    conv2.weight = torch.nn.Parameter(conv2w)
    conv2.bias = torch.nn.Parameter(conv2b)

    xj = F.relu(conv1(xj))
    xj = F.relu(conv2(xj))
    xj = nn.MaxPool2d(2)(xj)
    xj = xj.reshape((-1, 4*14*14))
    xj = F.relu(xj@fc1w + fc1b)
    xj = xj@fc2w + fc2b
    ypred[j]=torch.softmax(xj,1)
  return - ypred[torch.arange(0, ypred.shape[0]), ytrue.long()].log().mean()

In [50]:
print(cross_entropycw1(conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b,ytrain, Xtrain))
print(cross_entropycb1(conv1b, conv1w, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b,ytrain, Xtrain))
print(cross_entropycw2(conv2w, conv1b, conv1w, conv2b, fc1w, fc1b, fc2w, fc2b,ytrain, Xtrain))
print(cross_entropycb2(conv2b, conv1b, conv2w, conv1w, fc1w, fc1b, fc2w, fc2b,ytrain, Xtrain))
print(cross_entropyfcw1(fc1w, conv1b, conv2w, conv2b, conv1w, fc1b, fc2w, fc2b,ytrain, Xtrain))
print(cross_entropyfcb1(fc1b, conv1b, conv2w, conv2b, fc1w, conv1w, fc2w, fc2b,ytrain, Xtrain))
print(cross_entropyfcw2(fc2w, conv1b, conv2w, conv2b, fc1w, fc1b, conv1w, fc2b,ytrain, Xtrain))
print(cross_entropyfcb2(fc2b, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, conv1w,ytrain, Xtrain))

tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)
tensor(23.8395, grad_fn=<NegBackward0>)


Train and evaluate the network with forward gradient

In [51]:
def accuracy(ytrue, ypred):
  return (ypred.argmax(1) == ytrue).float().mean()

In [53]:
# Average accuracy at initialization is 10% (random guessing).
accuracy(ytrain[0].to(device), cnn(Xtrain[0].to(device)))

tensor(0., device='cuda:0')

In [54]:
def beale_function(x):
  return (torch.pow(torch.tensor([1.5])-x[0]+x[0]*x[1],2) + torch.pow(torch.tensor([2.25])-x[0]+x[0]*torch.pow(x[1],2),2)+torch.pow(torch.tensor([2.625])-x[0]+x[0]*torch.pow(x[1],3),2))

In [55]:
def rosenbrock_function(x):
  sum=0
  for p in x.size():
    for i in range (x.size(1)-1):
      sum += (100*torch.pow(x[i+1] - torch.pow(x[i], 2), 2) + torch.pow(x[i]-1, 2))
  return sum

In [57]:
from functorch import jvp
from functools import partial
import time 

In [64]:
def train_fwd_gradient(x, y):
  #x, y = x.to(device), y.to(device)

  l_rate0 = 0.005

  #Parameters
  conv1w = torch.FloatTensor(2, 1, 3, 3).uniform_(-1, 1)
  conv1b = torch.FloatTensor(2).uniform_(-1, 1)
  conv2w = torch.FloatTensor(4, 2, 3, 3).uniform_(-1, 1)
  conv2b = torch.FloatTensor(4).uniform_(-1, 1)
  fc1w = torch.FloatTensor(4*14*14,8).uniform_(-1,1)
  fc1b = torch.FloatTensor(8).uniform_(-1, 1)
  fc2w = torch.FloatTensor(8,10).uniform_(-1, 1)
  fc2b = torch.FloatTensor(10).uniform_(-1, 1)

  loss = cross_entropycw1(conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b, ytrain, Xtrain)

  t=0
  t0 = time.time()

  while (loss>0.6) :

    vconv1w=torch.randn(conv1w.shape)
    vconv1w=(vconv1w - torch.mean(vconv1w))/torch.std(vconv1w)
    vconv1b=torch.randn(conv1b.shape)
    vconv1b=(vconv1b - torch.mean(vconv1b))/torch.std(vconv1b)
    vconv2w=torch.randn(conv2w.shape)
    vconv2w=(vconv2w - torch.mean(vconv2w))/torch.std(vconv2w)
    vconv2b=torch.randn(conv2b.shape)
    vconv2b=(vconv2b - torch.mean(vconv2b))/torch.std(vconv2b)
    vfc1w=torch.randn(fc1w.shape)
    vfc1w=(vfc1w - torch.mean(vfc1w))/torch.std(vfc1w)
    vfc1b=torch.randn(fc1b.shape)
    vfc3b=(vfc1b - torch.mean(vfc1b))/torch.std(vfc1b)
    vfc2w=torch.randn(fc2w.shape)
    vfc2w=(vfc2w - torch.mean(vfc2w))/torch.std(vfc2w)
    vfc2b=torch.randn(fc2b.shape)
    vfc2b=(vfc2b - torch.mean(vfc2b))/torch.std(vfc2b)

    fw1 = partial(cross_entropycw1, conv1b=conv1b, conv2w=conv2w, conv2b=conv2b, fc1w=fc1w, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b,ytrue=y, x=x)
    fb1 = partial(cross_entropycb1, conv1w=conv1w, conv2w=conv2w, conv2b=conv2b, fc1w=fc1w, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b,ytrue=y, x=x)
    fw2 = partial(cross_entropycw2, conv1b=conv1b, conv1w=conv1w, conv2b=conv2b, fc1w=fc1w, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b,ytrue=y, x=x)
    fb2 = partial(cross_entropycb2, conv1b=conv1b, conv2w=conv2w, conv1w=conv1w, fc1w=fc1w, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b,ytrue=y, x=x)
    fcw1 = partial(cross_entropyfcw1, conv1b=conv1b, conv2w=conv2w, conv2b=conv2b, conv1w=conv1w, fc1b=fc1b, fc2w=fc2w, fc2b=fc2b,ytrue=y, x=x)
    fcb1 = partial(cross_entropyfcb1, conv1b=conv1b, conv2w=conv2w, conv2b=conv2b, fc1w=fc1w, conv1w=conv1w, fc2w=fc2w, fc2b=fc2b,ytrue=y, x=x)
    fcw2 = partial(cross_entropyfcw2, conv1b=conv1b, conv2w=conv2w, conv2b=conv2b, fc1w=fc1w, fc1b=fc1b, conv1w=conv1w, fc2b=fc2b,ytrue=y, x=x)
    fcb2 = partial(cross_entropyfcb2, conv1b=conv1b, conv2w=conv2w, conv2b=conv2b, fc1w=fc1w, fc1b=fc1b, fc2w=fc2w, conv1w=conv1w,ytrue=y, x=x)

    loss, dtw1=jvp(fw1,(conv1w, ), (vconv1w, ))
    print('dtw1', dtw1)
    ftb1, dtb1=jvp(fb1,(conv1b, ), (vconv1b, ))
    ftw2, dtw2=jvp(fw2,(conv2w, ), (vconv2w, ))
    ftb2, dtb2=jvp(fb2,(conv2b, ), (vconv2b, ))
    ftfcw1, dtfcw1=jvp(fcw1,(fc1w, ), (vfc1w, ))
    print('dtfcw1', dtfcw1)
    ftfcb1, dtfcb1=jvp(fcb1,(fc1b, ), (vfc1b, ))
    ftfcw2, dtfcw2=jvp(fcw2,(fc2w, ), (vfc2w, ))
    ftfcb2, dtfcb2=jvp(fcb2,(fc2b, ), (vfc2b, ))

    gtw1 = vconv1w*dtw1
    gtb1 = vconv1b*dtb1
    gtw2 = vconv2w*dtw2
    gtb2 = vconv2b*dtb2
    gtfcw1 = vfc1w*dtfcw1
    gtfcb1 = vfc1b*dtfcb1
    gtfcw2 = vfc2w*dtfcw2
    gtfcb2 = vfc2b*dtfcb2

    conv1w -= l_rate0*gtw1
    conv1b -= l_rate0*gtb1
    conv2w -= l_rate0*gtw2
    conv2b -= l_rate0*gtb2
    fc1w -= l_rate0*gtfcw1
    fc1b -= l_rate0*gtfcb1
    fc2w -= l_rate0*gtfcw2
    fc2b -= l_rate0*gtfcb2

    t1=time.time()
    t+=t1-t0
    t0=t1
    print('Time', t, 'loss', loss)

  return conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b

In [65]:
conv1w, conv1b, conv2w, conv2b, fc1w, fc1b, fc2w, fc2b = train_fwd_gradient(Xtrain, ytrain)

dtw1 tensor(0.)
dtfcw1 tensor(2.9895)
Time 4.269709587097168 loss tensor(12.8529)
dtw1 tensor(0.)
dtfcw1 tensor(-0.3732)
Time 8.188392162322998 loss tensor(12.7976)
dtw1 tensor(0.)
dtfcw1 tensor(-3.3361)
Time 12.895508766174316 loss tensor(12.5891)
dtw1 tensor(0.)
dtfcw1 tensor(-0.4618)
Time 16.81680989265442 loss tensor(12.5363)
dtw1 tensor(0.)
dtfcw1 tensor(5.5722)
Time 20.868961811065674 loss tensor(12.5349)
dtw1 tensor(0.)
dtfcw1 tensor(14.0841)
Time 25.663103103637695 loss tensor(12.3993)
dtw1 tensor(0.)
dtfcw1 tensor(-0.4268)
Time 29.767574787139893 loss tensor(11.7671)
dtw1 tensor(0.)
dtfcw1 tensor(0.2991)
Time 33.83362531661987 loss tensor(11.6618)
dtw1 tensor(0.)
dtfcw1 tensor(-11.3290)
Time 38.66539168357849 loss tensor(11.5428)
dtw1 tensor(0.)
dtfcw1 tensor(7.5732)
Time 42.66342878341675 loss tensor(10.9733)
dtw1 tensor(0.)
dtfcw1 tensor(-2.7177)
Time 46.683427810668945 loss tensor(10.7294)
dtw1 tensor(0.)
dtfcw1 tensor(-2.5825)
Time 51.477702379226685 loss tensor(10.6875)
d

KeyboardInterrupt: ignored